In [2]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import random

In [62]:
train_labels_path = '../data/raw/train/labels/'
train_images_path = '../data/raw/train/images/'
test_images_path = '../data/raw/test/images/'

interim_labels_path = '../data/interim/train/labels/'

yolo_labels = '../data/data_for_yolo/data/labels/'

class_names = {'human':3, 'head':1, 'face':2, 'car':0, 'carplate':4}

In [7]:
# Количество изображений для тестирования
len(os.listdir(test_images_path))

855

In [5]:
# Количество изображений для обучения
len(os.listdir(train_images_path))

2002

In [6]:
#распределение объектов в train
pd.Series([file_name.split('_')[-1] for file_name in os.listdir(train_labels_path)]).value_counts()

head..txt        1741
human..txt       1740
car..txt         1647
face..txt         630
carplate..txt     246
dtype: int64

In [8]:
#в папке data_for_yolo создадим dataset.yaml со следующим содержимым

yaml_content = """
train: ../data/data_for_yolo/data/images/train/
val: ../data/data_for_yolo/data/images/test/

# number of classes
nc: 5

# class names
names: ['car', 'head', 'face', 'human', 'carplate']
"""

with open('../data/data_for_yolo/dataset.yaml', 'w') as f:
    f.write(yaml_content)

# Подготовка данных

In [68]:
!mkdir ../data/interim
!mkdir ../data/interim/train
!mkdir ../data/interim/train/labels/

mkdir: невозможно создать каталог «../data/interim»: Файл существует


In [88]:
# Объединение разметки для каждого изображения в один файл

for file in os.listdir(train_images_path):
    
    file_base = os.path.splitext(file)[0]
    
    file_labels = [file_name for file_name in os.listdir(train_labels_path) 
                   if file_base == '_'.join(file_name.split('_')[:-1])]
    if len(file_labels) == 0:
        print('File: {file} not contain lables')
        continue
    
    # open file for write labels
    with open(os.path.join(interim_labels_path, file_base + '.txt'), 'w') as file_label:
        
        # process labels files
        for file in file_labels:
            label = file.split('.')[0].split('_')[-1]
            cls = class_names[label]

            # read labels info and write it to result label file
            with open(train_labels_path + file, 'r') as f:
                for line in f:
                    data = line.split(' ')
                    data[0] = cls
                    data = ' '.join(str(e) for e in data)
                    file_label.write(data)